In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('/kaggle/input/heart-disease-cleveland/Heart_disease_cleveland_new.csv')

In [3]:
X = data.drop('target', axis=1).astype(float)
y = data['target'].astype(int)

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [6]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.int64)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.int64)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.int64)

In [7]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [8]:
class HeartDiseaseClassifier(nn.Module):
    def __init__(self):
        super(HeartDiseaseClassifier, self).__init__()
        self.fc1 = nn.Linear(13, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # Output size is 2 for binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
model = HeartDiseaseClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
best_val_loss = float('inf')
best_model_path = 'best_model.pt'

In [11]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
        val_loss /= len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)
        print("Saving best model...")

Epoch [1/50], Validation Loss: 0.6792
Saving best model...
Epoch [2/50], Validation Loss: 0.6667
Saving best model...
Epoch [3/50], Validation Loss: 0.6531
Saving best model...
Epoch [4/50], Validation Loss: 0.6386
Saving best model...
Epoch [5/50], Validation Loss: 0.6268
Saving best model...
Epoch [6/50], Validation Loss: 0.6144
Saving best model...
Epoch [7/50], Validation Loss: 0.6011
Saving best model...
Epoch [8/50], Validation Loss: 0.5867
Saving best model...
Epoch [9/50], Validation Loss: 0.5712
Saving best model...
Epoch [10/50], Validation Loss: 0.5522
Saving best model...
Epoch [11/50], Validation Loss: 0.5333
Saving best model...
Epoch [12/50], Validation Loss: 0.5167
Saving best model...
Epoch [13/50], Validation Loss: 0.5030
Saving best model...
Epoch [14/50], Validation Loss: 0.4944
Saving best model...
Epoch [15/50], Validation Loss: 0.4863
Saving best model...
Epoch [16/50], Validation Loss: 0.4790
Saving best model...
Epoch [17/50], Validation Loss: 0.4712
Saving bes

In [12]:
model.load_state_dict(torch.load(best_model_path))

<All keys matched successfully>

In [13]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {(100 * correct / total):.2f}%')

Accuracy on test set: 86.89%
